In [7]:

%pip install pyspark
%pip install sentence-transformers
%pip install requests
%pip install tensorflow
%pip install tensorflow tensorflowjs
%pip install imbalanced-learn
%pip install scikit-learn==1.2.2
%pip install ipython
%pip install numpy==1.26.0
%pip install flask


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached scikit_learn-1.6.1-cp310-cp310

In [16]:
import json
import numpy as np
from flask import Flask, jsonify, request
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import requests
from tensorflow.keras.preprocessing import image
from io import BytesIO
from tensorflow.keras.utils import register_keras_serializable
import tensorflow.keras.backend as K

In [17]:
# Cargar datos de los animales y los usuarios
file_path_Animal = 'https://matchpet-e9757-default-rtdb.firebaseio.com/Animal.json'
file_path_User = 'https://matchpet-e9757-default-rtdb.firebaseio.com/User.json'

# Solicitar datos de Firebase
response_animal = requests.get(file_path_Animal)
response_user = requests.get(file_path_User)

if response_animal.status_code == 200 and response_user.status_code == 200:
    animal_data = response_animal.json()
    user_data = response_user.json()
else:
    print("Error al cargar los datos desde Firebase")
    animal_data = {}
    user_data = {}

# Función para codificar variables categóricas
def encode_categorical_variable(variable):
    label_encoder = LabelEncoder()
    return label_encoder.fit_transform(variable)

# Preprocesamiento de datos de animales
def preprocess_animal_data(animal_data):
    features = []
    for animal in animal_data.values():
        edad = int(animal["edad"])
        raza = encode_categorical_variable([animal["raza"]])[0]
        sexo = encode_categorical_variable([animal["sexo"]])[0]
        recomendation = encode_categorical_variable([animal["recomendation"]])[0]
        description = encode_categorical_variable([animal["descripcion"]])[0]

        features.append([edad, raza, sexo, recomendation, description])
    return np.array(features)

# Preprocesamiento de datos de usuarios
def preprocess_user_preferences(user_data, user_id):
    user_info = user_data.get(user_id, {})

    if 'preferences' not in user_info:
        print(f"El usuario {user_id} no tiene preferencias.")
        return None

    prefs = user_info['preferences']

    aceptacionContrato = 1 if prefs["aceptacionContrato"][0] == 'Sí' else 0
    compromisoEconomico = 1 if prefs["compromisoEconomico"][0] == 'Sí' else 0
    disponibilidadSeguimiento = 1 if prefs["disponibilidadSeguimiento"][0] == 'Sí' else 0
    espacioExterior = 1 if prefs["espacioExterior"][0] == 'Sí' else 0
    experienciaMascota = 1 if prefs["experienciaMascota"][0] == 'Sí' else 0

    foods = encode_categorical_variable([prefs["foods"][0]])[0]
    hobbies = encode_categorical_variable([prefs["hobbies"][0]])[0]
    moods = encode_categorical_variable([prefs["moods"][0]])[0]
    motivacion = encode_categorical_variable([prefs["motivacion"][0]])[0]
    preferenciaRaza = encode_categorical_variable([prefs["preferenciaRaza"][0]])[0]
    selectedLifestyles = encode_categorical_variable([prefs["selectedLifestyles"][0]])[0]
    situacionVivienda = encode_categorical_variable([prefs["situacionVivienda"][0]])[0]
    sports = encode_categorical_variable([prefs["sports"][0]])[0]
    tiempoFuera = encode_categorical_variable([prefs["tiempoFuera"][0]])[0]
    tipoMascota = encode_categorical_variable([prefs["tipoMascota"][0]])[0]
    ultimaMascota = encode_categorical_variable([prefs["ultimaMascota"][0]])[0]

    return np.array([[aceptacionContrato, compromisoEconomico, disponibilidadSeguimiento, espacioExterior,
                      experienciaMascota, foods, hobbies, moods, motivacion, preferenciaRaza,
                      selectedLifestyles, situacionVivienda, sports, tiempoFuera, tipoMascota, ultimaMascota]])

# Preprocesamiento de imágenes de los animales
def preprocess_image(img_path):
    try:
        response = requests.get(img_path)
        if response.status_code == 200:
            img = image.load_img(BytesIO(response.content), target_size=(224, 224))
            img_array = image.img_to_array(img) / 255.0
            return np.expand_dims(img_array, axis=0)
    except:
        return np.zeros((1, 224, 224, 3))

animal_features = preprocess_animal_data(animal_data)

# Definir la función de similaridad antes de cargar el modelo
@register_keras_serializable()
def compute_similarity(vectors):
    user_vec, animal_vec = vectors
    return K.exp(-K.sum(K.abs(user_vec - animal_vec), axis=1, keepdims=True))

# Cargar el modelo entrenado
model_path = 'Model/Best_Model_Siamese.keras'
model = load_model(model_path, custom_objects={'compute_similarity': compute_similarity})

# Función para predecir compatibilidad
def predict_compatibility(user_id, model, animal_data, user_data):
    # Preprocesar las preferencias del usuario
    user_features = preprocess_user_preferences(user_data, user_id)
    if user_features is None:
        return None

    # Preprocesar las imágenes de los animales
    animal_images = np.array([preprocess_image(animal["imagenes"][0]) for animal in animal_data.values()])
    animal_images = np.squeeze(animal_images, axis=1)

    # Expandir las características del usuario y los animales
    num_animals = len(animal_data)
    user_features_expanded = np.tile(user_features, (num_animals, 1))  # Expandir las características del usuario
    animal_features_expanded = np.array(animal_features)

    # Redimensionar para que la forma coincida con la esperada por el modelo (771, 1)
    user_features_expanded = np.concatenate([user_features_expanded, np.zeros((num_animals, 1165 - user_features_expanded.shape[1]))], axis=-1)
    animal_features_expanded = np.concatenate([animal_features_expanded, np.zeros((num_animals, 771 - animal_features_expanded.shape[1]))], axis=-1)

    user_features_expanded = np.expand_dims(user_features_expanded, axis=-1)
    animal_features_expanded = np.expand_dims(animal_features_expanded, axis=-1)

    # Realizar la predicción para todos los animales
    similarity_scores = model.predict([user_features_expanded, animal_features_expanded])

    # Retornar los resultados
    results = []
    for i, score in enumerate(similarity_scores):
        animal_id = list(animal_data.keys())[i]  # Obtener el ID del animal
        # Convertir la puntuación de compatibilidad a porcentaje
        compatibility_percentage = score[0] * 100
        results.append((animal_id, compatibility_percentage))  # Guardar el ID del perro y su puntuación de compatibilidad en porcentaje

    # Ordenar los resultados por la puntuación de compatibilidad
    results = sorted(results, key=lambda x: x[1], reverse=True)

    return results

# Ejemplo de uso: Predecir compatibilidad para un usuario dado
user_id = 'YN41Eq0ObQU9OoepbDEDxnNPN4D2'  # ID del usuario
compatibility_results = predict_compatibility(user_id, model, animal_data, user_data)

# Mostrar los resultados
if compatibility_results:
    print(f"Resultados de compatibilidad para el usuario {user_id}:")
    for animal_id, compatibility_percentage in compatibility_results:
        print(f"Animal ID: {animal_id}, Puntuación de compatibilidad: {compatibility_percentage:.2f}%")
else:
    print(f"No se encontraron resultados para el usuario {user_id}.")

/opt/homebrew/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
Resultados de compatibilidad para el usuario YN41Eq0ObQU9OoepbDEDxnNPN4D2:
Animal ID: -OEQUrHY4-_a36qYF1Cz, Puntuación de compatibilidad: 99.68%
Animal ID: -OEQWWGQzUWiSxBJrGag, Puntuación de compatibilidad: 99.68%
Animal ID: -OEM8Q08KydezI2uO5ct, Puntuación de compatibilidad: 99.65%
Animal ID: -OEM9j1EY7jlhmAheO0H, Puntuación de compatibilidad: 99.65%
Animal ID: -OEMBPqfP_Og3wAB4g6Y, Puntuación de compatibilidad: 99.65%
Animal ID: -OEPmnT5xJbq-OE-sfN6, Puntuación de compatibilidad: 99.65%
Animal ID: -OEQVDh4MzUKsf-r3YW7, Puntuación de compatibilidad: 99.65%
Animal ID: -OEQWE6jsbgr6xUCPozn, Puntuación de compatibilidad: 99.65%
Animal ID: -OERN1Z5B7kXZpGSoGA-, Puntuación de compatibilidad: 99.65%
Animal ID: -OEM4B9TzVo2Ap5ilLMV, Puntuación de compatibilidad: 99.54%
Animal ID: -OEMDXB5Uaf_9vQTHtqm, Puntuación de compatibilidad: 99.54%
Animal ID: -OEMC7kF4TpQ06LNIKD9, Puntuación de compatibilidad: 99.41%
Animal ID: -OEMD4G-lC8exqM7bxbl, Puntuación de